<a href="https://colab.research.google.com/github/jjangmo91/Cervus-nippon_Anmado-Is./blob/main/Cervus-nippon_Anmado-Is.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Earth Engine을 활용한 안마도 식생 분석
Google Earth Engine과 Colab을 이용하여, 안마도의 유기된 대만꽃사슴의 증가와 그로 인한 피해를 조사하는 프로젝트입니다.
안마도에 사슴이 들어온 1985년부터, 30여년간의 피해 상황을 시계열 분석을 통하여 식생 변화를 알아보겠습니다.
<br><br><br>
분석 단계는 다음과 같습니다:
* GEE의 Python API를 사용하여 안마도 영역을 설정합니다.
* 30년여간의 위성 데이터(Landsat)를 수집합니다.
* Landsat 데이터를 활용하여 구름 없는 대기 보정 이미지를 생산합니다.
* 다년도 이미지를 사용하여 NDVI 계산을 수행하고, 안마도의 NDVI 통계치를 계산해 봅니다.

## 1. Google Earth Engine 초기화 및 필요 라이브러리 로드


In [3]:
# Earth Engine Python API 모듈 및 라이브러리 호출
import ee
import geemap
import pandas as pd

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-jjangmo91')

## 2. 관심 영역(AOI) 설정
안마도의 위치를 관심 영역(AOI: Area of Interest)으로 설정합니다.

In [4]:
# 안마도의 위치를 기반으로 관심 지역을 설정합니다.
anmado_location = ee.Geometry.Point([126.028111, 35.344031])
AOI = anmado_location.bounds().buffer(distance=5000, maxError=100)

# 연구지역 경계를 지도에 추가
outline = ee.Image().byte().paint(featureCollection=AOI, color=1, width=3)

# 지도 객체 생성
Map = geemap.Map(center=[35.344031, 126.028111], zoom=12)

Map.addLayer(outline, {'palette': 'FF0000'}, "Study Area")
Map.centerObject(AOI, 7)
Map

Map(center=[35.34403612944367, 126.02811117438931], controls=(WidgetControl(options=['position', 'transparent_…

## 3. 위성 이미지 수집
위성 이미지 데이터를 시계열로 수집하여 분석합니다. 여기서는 Landsat5(TM), Landsat8(OLI/TIRS) 이미지를 사용합니다.
<br>
* Landsat5는 미국 지질 조사국(USGS)과 나사(NASA)가 공동으로 운영하는 Landsat 위성 프로그램의 일부로, 1984년 3월에 발사되어 2013년까지 약 29년 동안 운영되었습니다. 전 세계를 대상으로 16일마다 재방문하는 광역(wide-swath), 중간 해상도(medium-resolution) 다중 스펙트럼(multispectral) 이미징 임무를 수행했습니다. 이 위성의 Thematic Mapper(TM) 센서는 7개의 분광 밴드를 포함하며, 가시광선 및 근적외선(NIR)을 30미터, 열적외선을 120미터(재조정으로 30미터) 공간 해상도로 측정합니다. 이를 통해 식생, 토양, 수체의 상태 및 변화를 평가하는 데 적합한 데이터를 제공합니다.

* Landsat8은 2013년에 발사되었으며, Landsat5와 마찬가지로 16일마다 재방문하는
광역(wide-swath), 중간 해상도(medium-resolution), 다중 스펙트럼 이미징 임무(multispectral imaging mission)입니다. Landsat8의 두 주요 센서인 Operational Land Imager(OLI)와 Thermal Infrared Sensor(TIRS)는 다양한 분광 밴드를 취급합니다. OLI는 가시광선 및 근적외선(NIR)을 30미터, 해안/에어로졸 밴드를 30미터 해상도로, TIRS는 두 개의 열적외선 밴드를 통해 100미터 해상도로 측정합니다. 이는 지표 온도, 식생, 토양, 수체의 상태 및 변화를 평가하는 데 적합한 데이터를 제공하여 농업, 산림, 도시 계획 및 환경 관리 등에 광범위하게 활용됩니다.
<br>
아래의 함수는 pixel_qa 레이어의 비트마스크를 사용하여 구름, 구름 그림자, 눈이 있는 픽셀을 식별하고 제거합니다. 이렇게 처리된 이미지는 맑은 조건의 지표면 반사율 정보만을 포함하게 되어, NDVI와 같은 지표를 계산할 때 오류의 가능성을 줄여 줍니다. 비트 마스크의 각 값은 구름은 5번, 구름 그림자는 3번, 눈은 4번 비트를 사용합니다. 마지막으로, 대기 보정된 데이터를 스케일링하기 위해 10000으로 나누어 정규화된 반사율 값을 얻습니다. 이 함수를 사용하여 Landsat 5 및 8 이미지 컬렉션에 적용하고, NDVI 계산과 같은 후속 분석을 수행할 수 있습니다.



In [5]:
def maskLandsatClouds(image):
    # pixel_qa 레이어 선택
    qa = image.select('pixel_qa')

    # 비트 마스킹을 통해 구름(clouds), 구름 그림자(cloud shadows), 눈(snow) 식별
    cloud = qa.bitwiseAnd(1 << 5)
    cloudShadow = qa.bitwiseAnd(1 << 3)
    snow = qa.bitwiseAnd(1 << 4)

    # 구름, 그림자, 눈이 없는 맑은 상태의 이미지 마스크 생성
    mask = cloud.eq(0).And(cloudShadow.eq(0)).And(snow.eq(0))

    # 마스크를 적용하고, 스케일 조정을 위해 결과를 10000으로 나눔
    return image.updateMask(mask).divide(10000)

Landsat 5 TM과 Landsat 8 OLI/TIRS 위성 이미지를 선택하고 필터링하는 과정을 수행합니다. 구체적으로 1985년부터 2024년 4월 28일까지의 기간 동안 촬영된 이미지를 대상으로 안마도 지역에 해당하는 이미지를 필터링합니다. 또한, 'pixel_qa' 레이어를 사용하여 구름과 그림자를 제거하여 맑은 상태의 이미지만을 추출하는 과정을 포함합니다.

In [8]:
# Landsat 5 TM과 Landsat 8 OLI/TIRS 이미지 선택 및 필터링
landsat5_images = (
    ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")
    .filterDate("1985-01-01", "2012-05-05")  # Landsat 5 운영 기간
    .filterBounds(AOI)
    .map(maskLandsatClouds)  # 구름 및 그림자 마스킹 함수 적용
)

landsat8_images = (
    ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
    .filterDate("2013-04-11", "2024-01-31")  # Landsat 8 운영 기간
    .filterBounds(AOI)
    .map(maskLandsatClouds)  # 구름 및 그림자 마스킹 함수 적용
)

# 각 이미지 컬렉션의 이미지 개수 확인
image_count_l5 = landsat5_images.size()
image_count_l8 = landsat8_images.size()

# 이미지 개수 출력
print("Landsat 5 image count:", image_count_l5.getInfo())
print("Landsat 8 image count:", image_count_l8.getInfo())

Landsat 5 image count: 716
Landsat 8 image count: 298


필터링된 이미지 컬렉션 내의 모든 이미지들로부터 밴드별 중간값을 계산하여 새로운 단일 이미지를 생성합니다. 이는 구름이 적은 여러 날짜의 이미지들을 통합하여 더 깨끗한 대표 이미지를 얻기 위해 사용됩니다. 이미지는 RGB 컬러로 표시합니다. Landsat5에서 B3, B2, B1는 각각 적색, 녹색, 청색 색상에 해당하며, Landsat8에서는 B4, B3, B2가 각각 적색, 녹색, 청색에 해당합니다.

In [11]:
# Landsat 5 TM 이미지 중간값 계산
landsat5_median = landsat5_images.median()

# Landsat 8 OLI/TIRS 이미지 중간값 계산
landsat8_median = landsat8_images.median()

# 시각화를 위한 매개변수 설정
visualization_l5 = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B3', 'B2', 'B1'],
}

visualization_l8 = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

# 지도 객체 생성
Map = geemap.Map(width="800px", height="500px")

# Landsat 5 TM 및 Landsat 8 OLI/TIRS 중간값 이미지를 지도에 추가
Map.add_layer(landsat5_median, visualization_l5, 'Landsat 5 RGB')
Map.add_layer(landsat8_median, visualization_l8, 'Landsat 8 RGB')

# 지도의 중심을 관심 지역(AOI)에 맞춤
Map.centerObject(AOI, 13)

# 지도 객체 출력
Map

Map(center=[35.34403612944367, 126.02811117438931], controls=(WidgetControl(options=['position', 'transparent_…

## 4. NDVI 계산 및 통계치 산출하기
프리즘을 통해 볼 수 있듯이, 태양광 스펙트럼은 많은 다른 파장으로 구성되어 있습니다. 태양광이 물체에 비추어질 때, 특정 파장은 흡수되고 다른 파장은 반사됩니다. 식물 잎의 색소인 클로로필(chlorophyll)은 광합성에 사용되는 가시광선을 강하게 흡수합니다. 반면에, 잎의 세포 구조는 근적외선을 강하게 반사합니다. 나무가 클로로필과 클로로필을 포함하는 잎을 더 많이 가질수록, 이러한 파장의 빛은 더 많이 영향을 받습니다. 과학자들은 식물이 빛과 상호작용하는 이 지식을 활용하여 지구 표면 전역의 식물이 흡수하고 반사하는 적색과 근적외선의 파장을 측정하기 위해 위성 센서를 설계함으로써 지구의 풍경을 통틀어 녹색 식생 밀도를 매핑합니다.

식물이 반사하는 적색 빛의 반사율을 근적외선 빛의 반사율에서 빼고, 그 차이를 적색과 근적외선 빛의 반사율의 합으로 나누면 과학자들이 정규식생지수(NDVI: Normalized Difference Vegetation Index)라고 부르는 값을 얻을 수 있습니다.

Landsat 5 TM 및 Landsat 8 OLI/TIRS 위성 이미지를 사용하여 NDVI를 계산하고 시각화하는 과정을 아래와 같이 설계할 수 있습니다:

Landsat 5 TM
Landsat 5 TM 위성에서는 B4가 근적외선(NIR) 밴드, B3가 적색(Red) 밴드입니다.

In [10]:
from ipyleaflet import TileLayer

# Vworld 배경지도 객체
vworld_base = TileLayer(
    url='https://xdworld.vworld.kr/2d/Base/service/{z}/{x}/{y}.png',
    name='Vworld Base',
    attribution='Vworld',
)

# Landsat 5 TM 이미지에서 NDVI 계산
ndvi_l5 = landsat5_images.median().normalizedDifference(['B4', 'B3'])

# NDVI 색상 팔레트 정의
ndvi_palette = [
    'FE8374',  # 낮은 NDVI - 갈색
    'FED976',  # 낮은-중간 NDVI - 밝은 녹색
    'CAE23C',  # 중간 NDVI - 녹색
    '98B718',  # 중간-높은 NDVI - 진한 녹색
    '059033',  # 높은 NDVI - 매우 진한 녹색
]

# Vworld 하이브리드지도 객체
vworld_hybrid = TileLayer(
    url='https://xdworld.vworld.kr/2d/Hybrid/service/{z}/{x}/{y}.png',
    name='Vworld Hybrid',
    attribution='Vworld',
)

# NDVI 시각화 및 지도에 추가
Map = geemap.Map(width="800px", height="500px")
Map.add_layer(ndvi_l5, {'min': 0, 'max': 0.5, 'palette': ndvi_palette}, 'NDVI Landsat 5')
Map.add_layer(vworld_hybrid)
Map.centerObject(AOI, 13) # 지도의 중심 설정
Map # 지도 객체 출력

Map(center=[35.34403612944367, 126.02811117438931], controls=(WidgetControl(options=['position', 'transparent_…

Landsat 8 OLI/TIRS
Landsat 8 OLI에서는 B5가 근적외선(NIR) 밴드, B4가 적색(Red) 밴드입니다.

In [12]:
# Landsat 8 OLI/TIRS 이미지에서 NDVI 계산
ndvi_l8 = landsat8_images.median().normalizedDifference(['B5', 'B4'])

# NDVI 시각화 및 지도에 추가
Map.add_layer(ndvi_l8, {'min': 0, 'max': 0.5, 'palette': ndvi_palette}, 'NDVI Landsat 8')

지도 구성 및 출력
지도에 두 NDVI 계층을 추가하고, 지도를 출력합니다.

In [ ]:
# 지도 객체 생성
Map = geemap.Map(width="800px", height="500px")
Map.add_layer(ndvi_l5, {'min': 0, 'max': 0.5, 'palette': ndvi_palette}, 'NDVI Landsat 5')
Map.add_layer(ndvi_l8, {'min': 0, 'max': 0.5, 'palette': ndvi_palette}, 'NDVI Landsat 8')
Map.centerObject(AOI, 13)  # 지도의 중심 설정
Map  # 지도 객체 출력

이 부분 포함해서 위에 과정에서 데이터 merge하고, 수정해야 함

1. 시계열 데이터 통합 및 분석
각 이미지의 NDVI를 계산한 후, 중간값을 이용해 장기간에 걸쳐 시계열 데이터를 통합합니다.
이를 통해 각 시간대별 대표적인 식생 상태를 나타내는 이미지를 생성합니다.


2. NDVI 통계치 산출
통합된 이미지에서 NDVI의 최소값, 최대값, 평균값, 중간값 등 표준편차를 계산합니다.
이 통계치는 식생 변화의 추세를 이해하는 데 중요합니다.


3. 결과 시각화 및 보고
계산된  NDVI 및 통계치를 사용하여 시간에 따른 식생 변화를 시각합니다.
안마도 생태계 변화를 모니터링하고, 장기적인 환경 변화 추세를 평가하는 데 사용됩니다.


평가


----------------------------------------------------------------------------



마지막으로, Sentinel-2 이미지를 기반으로 계산된 NDVI에 대한 다양한 통계치를 계산하고, 이를 데이터프레임으로 변환하여 CSV 파일로 저장하는 과정을 수행합니다.

reduceRegion 메서드를 사용하여 지정된 지역(AOI)에서 NDVI의 최소값, 평균값, 중간값, 최대값, 표준편차를 계산합니다. 이는 ee.Reducer 객체를 사용하여 여러 통계치를 결합함으로써 한 번의 연산으로 여러 통계치를 얻습니다. scale 파라미터는 해상도를 10m로 설정하며, maxPixels 파라미터는 처리할 최대 픽셀 수(1e9는 10억 개)를 지정합니다.

In [ ]:
# NDVI 통계치 계산 (최소값, 평균, 중간값, 최대값, 표준편차)
stats = ndvi.reduceRegion(
    reducer=ee.Reducer.min()
    .combine(reducer2=ee.Reducer.mean(), sharedInputs=True)
    .combine(reducer2=ee.Reducer.median(), sharedInputs=True)
    .combine(reducer2=ee.Reducer.max(), sharedInputs=True)
    .combine(reducer2=ee.Reducer.stdDev(), sharedInputs=True),
    geometry=AOI,
    scale=10,
    maxPixels=1e9,
)

# 통계치 결과를 DataFrame으로 변환
df_stats = pd.DataFrame(
    [stats.getInfo()],
    columns=["nd_min", "nd_mean", "nd_median", "nd_max", "nd_stdDev"],
    index=["AOI"],
)
df_stats.columns = ["Min", "Mean", "Median", "Max", "StdDev"]

# DataFrame을 CSV 파일로 저장하기
df_stats.to_csv('df_stats.csv', index=True)

# NDVI 통계치 출력
print(df_stats)

          Min      Mean    Median      Max    StdDev
AOI -0.580742 -0.339951 -0.402324  0.80899  0.207272
